In [1]:
import pygsheets
from google.cloud import secretmanager
from google.cloud import bigquery
from datetime import datetime 
from datetime import timedelta

import requests
import os
import urllib
import google.oauth2.id_token
import google.auth.transport.requests
import subprocess
import pygsheets
from google.cloud import secretmanager

from credentials import PROJECT_ID, SM_SECRET_NAME, SHEET_KEY_HEIRLOOM_PRICING_MODEL, SHEET_TAB_COMPLIANCE_TOGGLE, CF_GET_TOKEN_V2_URL

In [3]:
# ------ SECRET MANAGER ---------
class SecretManager():
    """ Construct Secret Manager client
    """
    client = secretmanager.SecretManagerServiceClient()

    def __init__(self, project_id):
        self.project_id = project_id
        self.parent = f"projects/{project_id}"

    def get_secret(self, secret_id, version_id:str = "latest"):
        """ Get secret value

            TODO:
                - Raise if secret not found
            Args:
                secret_id (str): Secret ID
                version_id (str): Secret version ID
                    default: latest
            Returns:
                secret_value (str): Secret value
        """
        # Access the secret version
        response = self.client.access_secret_version(
            request={
                "name": f"projects/{self.project_id}/secrets/{secret_id}/versions/{version_id}"
                })
        print("Accessed secret version: {}".format(response.name))
        # Return the decoded payload
        return response.payload.data.decode("UTF-8")     

c:\Users\Electro Store\AppData\Local\Programs\Python\Python39\lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


### Fetching data from Google Sheets

In [4]:
# Get the SA key from Secret Manager
sm = SecretManager(PROJECT_ID)
sa_key_json = sm.get_secret(SM_SECRET_NAME)

Accessed secret version: projects/800706532445/secrets/sa-key-pricing-pipeline/versions/1


In [5]:
gc = pygsheets.authorize(service_account_json=sa_key_json)
sheet_pricingModel = gc.open_by_key(SHEET_KEY_HEIRLOOM_PRICING_MODEL)
sheet_complianceToggle = sheet_pricingModel.worksheet_by_title(SHEET_TAB_COMPLIANCE_TOGGLE)
df_complianceToggle = sheet_complianceToggle.get_as_df()
df_complianceToggle

,Listing Unit Address,Timezone,Time In,Time Out,Number of bedrooms (In),Number of Occupants (In),Number of bedrooms (Out),Number of Occupants (Out),Min nights (In),Min Nights (out)
0,109 Twelve Oaks,EDT,7:00,18:00,,12,,23,,


### Fetching data from BigQuery

In [6]:
bq_client = bigquery.Client(project=PROJECT_ID)

c:\Users\Electro Store\AppData\Local\Programs\Python\Python39\lib\site-packages\google\auth\_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [7]:
query = bq_client.query("SELECT * FROM stayloom.Listings.t_listing_dictionary")
df_listing_dict = query.result().to_dataframe() 
df_listing_dict

,External_ID,Listing_Internal_ID,Unit_ID,Internal_Unit_Status,Date_of_Creation,Nickname_Input,Version_Input,Listied_on_any_channels_,Guesty_Nickname,Count,...,Entity_ID,Property_ID,HeyGPM_CC_COA_Property_Related_GL_,GPM_CC_COA_Property_Related_Account_Name,External_Reporting_,External_Reporting_Link,Neighborhood,Compliance_Sensitivity,Compliance_Classification__FOR_ANALYTICS___DO_NOT_USE_,Global_Min_Nights
0,59b8e30de11ac304005bf83d,1009V1,1009,Active,2017-09-13,23 Linwood 1,V1,False,23 Linwood 1 V1,15,...,L010,D000,18162,Due From Properties DND Enterprises LLC 23 Lin...,True,app.guesty.com/apps/reservations?view=6489e9d0...,Roxbury,<NA>,Non-compliant,1
1,5e25f90f3711ab0087404bf7,1009V2,1009,Active,2020-01-21,23 Linwood 1,V2,False,23 Linwood 1 V2,15,...,L010,D000,18162,Due From Properties DND Enterprises LLC 23 Lin...,True,app.guesty.com/apps/reservations?view=6489e9d0...,Roxbury,<NA>,Non-compliant,1
2,6077dd9362c7fc002f97d33b,1009V3,1009,Active,2021-07-20,23 Linwood 1,V3,False,23 Linwood 1 V3,15,...,L010,D000,18162,Due From Properties DND Enterprises LLC 23 Lin...,True,app.guesty.com/apps/reservations?view=6489e9d0...,Roxbury,<NA>,Non-compliant,1
3,618973dc4175ff002fd99e97,1009V4,1009,Active,2021-11-12,23 Linwood 1,V4,False,23 Linwood 1 V4,15,...,L010,D000,18162,Due From Properties DND Enterprises LLC 23 Lin...,True,app.guesty.com/apps/reservations?view=6489e9d0...,Roxbury,<NA>,Non-compliant,1
4,6489bcf82957770061ab2fb1,1009V5,1009,Active,2023-06-14,23 Linwood 1,V5,True,23 Linwood 1 V5,15,...,L010,D000,18162,Due From Properties DND Enterprises LLC 23 Lin...,True,app.guesty.com/apps/reservations?view=6489e9d0...,Roxbury,<NA>,Non-compliant,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1498,None,99995,99995,None,NaT,None,None,<NA>,None,<NA>,...,082,D000,<NA>,None,<NA>,None,None,<NA>,None,1
1499,None,99994,99994,None,NaT,None,None,<NA>,None,<NA>,...,001,D000,<NA>,None,<NA>,None,None,<NA>,None,1
1500,None,99993,99993,None,NaT,None,None,<NA>,None,<NA>,...,001,D000,<NA>,None,<NA>,None,None,<NA>,None,1
1501,None,99992,99992,None,NaT,None,None,<NA>,None,<NA>,...,001,D000,<NA>,None,<NA>,None,None,<NA>,None,1


### Analysis to Join data from both resources

In [8]:
# renaming column name in sheets df for join
df_complianceToggle.rename(columns={'Listing Unit Address': 'Listing_Unit_Address'}, inplace=True)

In [9]:
# listing_unit_address is the common column in both df
df_listingDict_listedTrue = df_listing_dict[['External_ID', 'Listing_Unit_Address']][df_listing_dict['Listied_on_any_channels_'] == True] # filtering the listings which are listed to get unique external id
df_listingDict_listedTrue

,External_ID,Listing_Unit_Address
4,6489bcf82957770061ab2fb1,23 Linwood 1
10,64beb6aad0f1e90046ed68a4,23 Linwood 2
18,6488714443a76f006ab9c1ea,23 Linwood 3
24,640bc0b60119f7003f641120,32 Buttonwood 1&2
27,64945ae30b3f6a0035aaf263,32 Buttonwood 2
...,...,...
900,65256487e4e873005805db44,11538 Bayou View
903,64d3ff6b8e1ce4003f963cfa,311 W Rosewood
966,564978d4971dfe03009a8f14,Test Listing
1154,5e29f9b32fc5e6001f07a910,8718 E Weldon


### Joining both df

In [10]:
df_joined = df_complianceToggle.merge(df_listingDict_listedTrue[['External_ID', 'Listing_Unit_Address']], on='Listing_Unit_Address')
df_joined

,Listing_Unit_Address,Timezone,Time In,Time Out,Number of bedrooms (In),Number of Occupants (In),Number of bedrooms (Out),Number of Occupants (Out),Min nights (In),Min Nights (out),External_ID
0,109 Twelve Oaks,EDT,7:00,18:00,,12,,23,,,64ee0e4855de32003d6318a7


### time zone mapping

|ABBREVIATION	|TIME ZONE NAME	                |UTC OFFSET
|---------------|-------------------------------|----------
|AST	        |ATLANTIC STANDARD TIME	        |UTC - 4
|EST	        |EASTERN STANDARD TIME	        |UTC - 5
|EDT	        |EASTERN DAYLIGHT TIME	        |UTC - 4
|CST	        |CENTRAL STANDARD TIME	        |UTC - 6
|CDT	        |CENTRAL DAYLIGHT TIME	        |UTC - 5
|MST	        |MOUNTAIN STANDARD TIME	        |UTC - 7
|MDT	        |MOUNTAIN DAYLIGHT TIME	        |UTC - 6
|PST	        |PACIFIC STANDARD TIME	        |UTC - 8
|PDT	        |PACIFIC DAYLIGHT TIME	        |UTC - 7
|AKST	        |ALASKA TIME	                |UTC - 9
|AKDT	        |ALASKA DAYLIGHT TIME	        |UTC - 8
|HST	        |HAWAII STANDARD TIME	        |UTC - 10
|HAST	        |HAWAII-ALEUTIAN STANDARD TIME	|UTC - 10
|HADT	        |HAWAII-ALEUTIAN DAYLIGHT TIME	|UTC - 9
|SST	        |SAMOA STANDARD TIME	        |UTC - 11
|SDT	        |SAMOA DAYLIGHT TIME	        |UTC - 10
|CHST	        |CHAMORRO STANDARD TIME	        |UTC +10

In [14]:
utc_now = datetime.utcnow() 
mapping_timezone = {
    'AST': (utc_now - timedelta(hours=4)).strftime('%H:%M'),
    'EST': (utc_now - timedelta(hours=5)).strftime('%H:%M'),
    'EDT': (utc_now - timedelta(hours=4)).strftime('%H:%M'),
    'CST': (utc_now - timedelta(hours=6)).strftime('%H:%M'),
    'CDT': (utc_now - timedelta(hours=5)).strftime('%H:%M'),
    'MST': (utc_now - timedelta(hours=7)).strftime('%H:%M'),
    'MDT': (utc_now - timedelta(hours=6)).strftime('%H:%M'),
    'PST': (utc_now - timedelta(hours=8)).strftime('%H:%M'),
    'PDT': (utc_now - timedelta(hours=7)).strftime('%H:%M'),
    'AKST': (utc_now - timedelta(hours=9)).strftime('%H:%M'),
    'AKDT': (utc_now - timedelta(hours=8)).strftime('%H:%M'),
    'HST': (utc_now - timedelta(hours=10)).strftime('%H:%M'),
    'HAST': (utc_now - timedelta(hours=10)).strftime('%H:%M'),
    'HADT': (utc_now - timedelta(hours=9)).strftime('%H:%M'),
    'SST': (utc_now - timedelta(hours=11)).strftime('%H:%M'),
    'SDT': (utc_now - timedelta(hours=10)).strftime('%H:%M'),
    'CHST': (utc_now + timedelta(hours=10)).strftime('%H:%M')
}
mapping_timezone

{'AST': '03:04',
 'EST': '02:04',
 'EDT': '03:04',
 'CST': '01:04',
 'CDT': '02:04',
 'MST': '00:04',
 'MDT': '01:04',
 'PST': '23:04',
 'PDT': '00:04',
 'AKST': '22:04',
 'AKDT': '23:04',
 'HST': '21:04',
 'HAST': '21:04',
 'HADT': '22:04',
 'SST': '20:04',
 'SDT': '21:04',
 'CHST': '17:04'}

### populating actual occupancy for each listing

In [11]:
df_joined['actual_occupancy'] = ''
df_joined

,Listing_Unit_Address,Timezone,Time In,Time Out,Number of bedrooms (In),Number of Occupants (In),Number of bedrooms (Out),Number of Occupants (Out),Min nights (In),Min Nights (out),External_ID,actual_occupancy
0,109 Twelve Oaks,EDT,7:00,18:00,,12,,23,,,64ee0e4855de32003d6318a7,


In [15]:
# function to apply logic of populating actual occupancy 
def set_actual_occupancy(df_row):
    timezone = df_row['Timezone'] # fetching timezone of a listing
    tz_time = mapping_timezone[timezone] # mapping timezone with its current time 
    time_in = df_row['Time In']
    time_out = df_row['Time Out']
    
    # if the current time of the listing's timezone lies between time in and time out value of that listing then 
    # Number of Occupants (In) value will be actual occupancy value
    if datetime.strptime(time_in, '%H:%M') <= datetime.strptime(tz_time, '%H:%M') <= datetime.strptime(time_out, '%H:%M'):
        return df_row['Number of Occupants (In)']
    else:
        return df_row['Number of Occupants (Out)']

In [16]:
df_joined['actual_occupancy'] = df_joined.apply(set_actual_occupancy, axis= 1)
df_joined

else


,Listing_Unit_Address,Timezone,Time In,Time Out,Number of bedrooms (In),Number of Occupants (In),Number of bedrooms (Out),Number of Occupants (Out),Min nights (In),Min Nights (out),External_ID,actual_occupancy
0,109 Twelve Oaks,EDT,7:00,18:00,,12,,23,,,64ee0e4855de32003d6318a7,23


### applying actual occupancy to Guesty listings

In [ ]:
# Get the Guesty API key
def get_token(url):
    """ Generate token and make request to endpoint

        Docs:
            - https://cloud.google.com/functions/docs/securing/authenticating#console
    """
    # Environment: C.Functions
    if "FUNCTION_TARGET" in os.environ:
        req = urllib.request.Request(url)
        auth_req = google.auth.transport.requests.Request()
        id_token = google.oauth2.id_token.fetch_id_token(auth_req, url)
        req.add_header("Authorization", f"Bearer {id_token}")
        response = urllib.request.urlopen(req)
        return response.read().decode("utf-8")
    # Environment: Local
    else:
        # HACK: Get the token from gcloud
        gcloud_itoken = subprocess.check_output(["gcloud","auth", "print-identity-token"], shell= True)
        token = gcloud_itoken.decode().strip()
        # Make request
        headers = {
            'Content-Type': 'application/json',
            'Authorization': f'Bearer {token}'
        }
        return requests.get(url, headers=headers).text

In [ ]:
token = get_token(CF_GET_TOKEN_V2_URL)
# print(token)

guesty_headers = {
    'Authorization': f'Bearer {token}',
    'content-type': 'application/json'
}

In [ ]:
# # function to populate actual occupancy (custom field) of each listing on Guesty
# def populating_actualOcc_toGuesty(df_row):
#     url_update_actualOcc = f"https://open-api.guesty.com/v1/listings/{df_row['External_ID']}/custom-fields"
#     payload = { "customFields": [
#             {
#                 "fieldId": "5ae01c0b34249a00248b2f62",
#                 "value": str(df_row['actual_occupancy'])
#             }
#         ] }
#     response = requests.put(url_update_actualOcc, json=payload, headers=guesty_headers)

#     print(f"Finished with status code: {response.status_code}")

In [ ]:
# function to populate accommodates (actual field) of each listing on Guesty
def populating_actualOcc_toGuesty(df_row):
    url_update_accomodate = f"https://open-api.guesty.com/v1/listings/{df_row['External_ID']}"
    payload = { "accommodates": str(df_row['actual_occupancy']) }
    response = requests.put(url_update_accomodate, json=payload, headers=guesty_headers)

    print(f"API responded with status code: {response.status_code}")

In [ ]:
df_joined.apply(populating_actualOcc_toGuesty, axis=1)